# Model Management
## Azure ML and MLFlow tracking

In [ ]:
# Connect to workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import mlflow

credential = DefaultAzureCredential()
aml_client = MLClient(
    credential=credential,
    subscription_id="49b5441f-dda4-47a9-81c4-13272430f4ff",
    resource_group_name="rg-pooya120-dev",
    workspace_name="amlwpooya120dev",
)
mlf_client = mlflow.tracking.MlflowClient()

In [ ]:
model_name = 'sklearn.ensemble._forest.RandomForestRegressor'

models = [x 
    for x in aml_client.models.list() 
    #if x.name==model_name
]

for i,m in enumerate(models):
    print(i, m.name, m.version)

## How to pick the right model among models?

In [ ]:
# for model in client.search_registered_models():
#     print(f"{model.name}")

mlf_client.search_registered_models(f"name='{model_name}'")
models = mlf_client.search_model_versions(f"name='{model_name}'")

m3 = mlf_client.get_model_version(model_name, 3) 
m2 = mlf_client.get_model_version(model_name, 2)
m1 = mlf_client.get_model_version(model_name, 1)

for m in [m1,m2,m3]:
    run = mlf_client.get_run(m.run_id)
    r2 = run.data.metrics['training_r2_score']
    print(m.version, r2)

In [ ]:
# find the model currently in production
endpoint_name = 'my-endpoint2'
deployment_name = 'blue2'
new_model_version = 3

from azure.ai.ml.entities import Model

def get_model_from_uri(uri):
    tokens = uri.split('/')
    tid = tokens.index('models')
    model = tokens[tid+1]
    vid = tokens.index('versions')
    version = tokens[vid+1]
    return aml_client.models.get(model,version)

def get_deployed_model(endpoint_name, deployment_name)->Model:
    endpoint = aml_client.online_endpoints.get(endpoint_name)
    deployment = aml_client.online_deployments.get(deployment_name, endpoint.name)
    deployed_model = get_model_from_uri(deployment.model)
    return deployed_model

def get_model_run(model: Model):
    client = mlflow.tracking.MlflowClient()
    return client.get_run(model.job_name)

def compare_models(old_model, new_model, metrics):
    old_run = get_model_run(old_model)
    new_run = get_model_run(new_model)
    for metric in metrics:
        print(f'- {metric}: {old_run.data.metrics[metric]}  vs {old_run.data.metrics[metric]}')

deployed_model = get_deployed_model(endpoint_name, deployment_name)


new_model = aml_client.models.get(model_name, new_model_version)

compare_models( deployed_model, new_model, [
    'training_mean_squared_error',
    'training_mean_absolute_error', 
    'training_r2_score', 
    'training_root_mean_squared_error', 
    'training_score'
])

In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)


In [ ]:
%%writefile score/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - azureml-defaults>=1.42.0
    - azureml-inference-server-http
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "bike-share-env"

env = Environment(
    name=custom_env_name,
    description="Custom environment for Bike Share pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file= "score/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
env = ml_client.environments.create_or_update(env)

print(
    f"Environment {env.name} registered to workspace. "
    f"Environment version is {env.version}."
)

In [ ]:
# Define an endpoint name
endpoint_name = "my-endpoint2"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Bike share endpoint",
    auth_mode="key",
)

ml_client.online_endpoints.begin_create_or_update(endpoint, ).result()

In [ ]:
model = ml_client.models.get('sklearn.ensemble._forest.RandomForestRegressor',1)
# env = ml_client.environments.get('aml-scikit-learn', 1)

deployment = ManagedOnlineDeployment(
    name="blue2",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./score", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    app_insights_enabled=True,
)
ml_client.online_deployments.begin_create_or_update(deployment=deployment, ).result()

In [ ]:
X,y = get_data()
Xe = encode_cols(X, ['workingday', 'weathersit'])

In [ ]:
Xe[0:10].to_numpy()
